<a href="https://colab.research.google.com/github/michaelscheinfeild/pytorch-transformer/blob/main/InferenceA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# transformer

transformer from scratch
https://www.youtube.com/watch?v=ISNdQcPhsts
https://github.com/hkproj/pytorch-transformer


In [1]:
!git clone https://github.com/hkproj/pytorch-transformer.git


Cloning into 'pytorch-transformer'...
remote: Enumerating objects: 140, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 140 (delta 43), reused 38 (delta 38), pack-reused 93
Receiving objects: 100% (140/140), 3.16 MiB | 7.62 MiB/s, done.
Resolving deltas: 100% (80/80), done.
/content/pytorch-transformer


In [ ]:
%cd pytorch-transformer


In [3]:
%ls

attention_visual.ipynb  config.py        Local_Train.ipynb  README.md
Beam_Search.ipynb       dataset.py       model.py           requirements.txt
Colab_Train.ipynb       Inference.ipynb  __pycache__/       train.py


In [4]:
from pathlib import Path
import torch
import torch.nn as nn


In [ ]:
!pip install datasets

In [ ]:
!pip install tokenizers

In [ ]:
!pip install torchmetrics

In [9]:
from config import get_config, get_weights_file_path

In [15]:
from train import get_model, get_ds, run_validation

In [16]:
# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [17]:
config = get_config()
config

{'batch_size': 8,
 'num_epochs': 20,
 'lr': 0.0001,
 'seq_len': 350,
 'd_model': 512,
 'lang_src': 'en',
 'lang_tgt': 'it',
 'model_folder': 'weights',
 'model_basename': 'tmodel_',
 'preload': None,
 'tokenizer_file': 'tokenizer_{0}.json',
 'experiment_name': 'runs/tmodel'}

In [18]:
train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(config)

Generating train split:   0%|          | 0/32332 [00:00<?, ? examples/s]

Max length of source sentence: 309
Max length of target sentence: 274


In [20]:
print(train_dataloader.dataset)

In [22]:
print(tokenizer_src.get_vocab_size())

15698


In [23]:
model = get_model(config, tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size()).to(device)

In [24]:
model

Transformer(
  (encoder): Encoder(
    (layers): ModuleList(
      (0-5): 6 x EncoderBlock(
        (self_attention_block): MultiHeadAttentionBlock(
          (w_q): Linear(in_features=512, out_features=512, bias=False)
          (w_k): Linear(in_features=512, out_features=512, bias=False)
          (w_v): Linear(in_features=512, out_features=512, bias=False)
          (w_o): Linear(in_features=512, out_features=512, bias=False)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (feed_forward_block): FeedForwardBlock(
          (linear_1): Linear(in_features=512, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear_2): Linear(in_features=2048, out_features=512, bias=True)
        )
        (residual_connections): ModuleList(
          (0-1): 2 x ResidualConnection(
            (dropout): Dropout(p=0.1, inplace=False)
            (norm): LayerNormalization()
          )
        )
      )
    )
    (norm): LayerNormalization

In [26]:
# Load the pretrained weights
model_filename = get_weights_file_path(config, f"29")
model_filename

'weights/tmodel_29.pt'

In [25]:

state = torch.load(model_filename)
model.load_state_dict(state['model_state_dict'])

FileNotFoundError: ignored

In [ ]:
run_validation(model, val_dataloader, tokenizer_src, tokenizer_tgt, config['seq_len'], device, lambda msg: print(msg), 0, None, num_examples=10)